In [1]:
%reload_ext autoreload
%autoreload 2

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

from glob import glob
import wave
import math

import numpy as np
from scipy import stats

import essentia.standard as es
import librosa
import pandas as pd

log_dir = "logs/emb/"

[   INFO   ] MusicExtractorSVM: no classifier models were configured by default


## Audio Directories 

DISCOTUBE = "/mnt/projects/discotube/discotube-2023-03/audio-new/audio/"
audio_directories = os.listdir(DISCOTUBE)
audio_directory = audio_directories[0]
mp4_paths = glob(os.path.join(DISCOTUBE, audio_directory, "*.mp4"))
mp4_path = mp4_paths[0]

In [2]:
audio_dir = "/home/oaraz/nextcore/afp/neural-audio-fp-dataset/music/test-dummy-db-100k-full/from_fma_large10k-30s_for_mini/"
audio_paths = glob(os.path.join(audio_dir, "*/*.wav"))
print(len(audio_paths))

9978


## Load Eval Results

In [13]:
test_seq_len='1 3 5 9 11 19'
test_seq_len = np.asarray(list(map(int, test_seq_len.split())))

In [18]:
def display_results(model_name, epoc, test_seq_len):
    emb_dir = os.path.join(log_dir, model_name, epoc)
    test_ids = np.load(os.path.join(emb_dir, "test_ids.npy"))
    raw_score = np.load(os.path.join(emb_dir, "raw_score.npy"))
    # print(test_ids.shape)
    # print(raw_score.shape)

    top1_exact, top1_near, top3_exact, top10_exact = np.split(raw_score, 4, axis=1)
    top1_exact_rate = 100. * np.mean(top1_exact, axis=0)
    top1_near_rate = 100. * np.mean(top1_near, axis=0)
    top3_exact_rate = 100. * np.mean(top3_exact, axis=0)
    top10_exact_rate = 100. * np.mean(top10_exact, axis=0)
    all_metrics = np.stack((top1_exact_rate, top1_near_rate, top3_exact_rate, top10_exact_rate), axis=0)
    df = pd.DataFrame(all_metrics, columns=test_seq_len, index=["top1_exact", "top1_near", "top3_exact", "top10_exact"])
    print(df)
    # return df

In [23]:
model_name = "replication"
epoc = "100"
print(model_name)
display_results(model_name, epoc, test_seq_len)
print()

model_name = "dB_normalized_1"
epoc = "100"
print(model_name)
display_results(model_name, epoc, test_seq_len)
print()

model_name = "essentia_mel"
epoc = "100"
print(model_name)
display_results(model_name, epoc, test_seq_len)
print()


replication
                1     3      5      9      11    19
top1_exact   75.85  92.0  96.70  98.55  98.95  99.6
top1_near    82.60  93.4  97.05  98.60  98.95  99.6
top3_exact   84.30  94.9  97.80  99.15  99.30  99.8
top10_exact  87.20  95.9  98.30  99.25  99.45  99.8

dB_normalized_1
                1      3      5      9      11     19
top1_exact   77.30  93.40  97.05  98.85  99.15  99.55
top1_near    82.60  94.05  97.40  98.95  99.20  99.65
top3_exact   85.20  96.25  98.05  99.35  99.55  99.85
top10_exact  88.05  96.55  98.45  99.50  99.65  99.90

essentia_mel
                1      3      5      9      11     19
top1_exact   76.55  93.45  97.25  99.05  99.20  99.60
top1_near    82.85  94.50  97.50  99.05  99.20  99.60
top3_exact   84.90  96.15  98.50  99.40  99.55  99.85
top10_exact  87.90  97.05  98.85  99.45  99.60  99.85



In [19]:
from eval.utils.print_table import PrintTable

In [ ]:
pt = PrintTable(scr=scr, test_seq_len=test_seq_len,
                row_names=['Top1 exact', 'Top1 near', 'Top3 exact','Top10 exact'])
pt.update_table((top1_exact_rate, top1_near_rate, top3_exact_rate, top10_exact_rate))